# Data Anonymiser & Early Data Preparation

In [10]:
import pandas as pd

In [11]:
df = pd.read_excel('E:/Colonoscopy/Colon_Data.xls', header = [6], ParseDates = True) # This is one way to import excel files. There are up to 20 different arguments you can specify in the argument

In [12]:
df.shape # a lot of data

(27296, 15)

In [13]:
df.info() # mostly pandas objects and some datetimes.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27296 entries, 0 to 27295
Data columns (total 15 columns):
Hospital Number                 27296 non-null object
NHS Number                      27198 non-null object
Patient Name                    27296 non-null object
Date of Birth                   27296 non-null datetime64[ns]
Procedure                       27296 non-null object
Endoscopist                     27296 non-null object
Procedure Date                  27296 non-null datetime64[ns]
Age                             27296 non-null int64
Indications                     27296 non-null object
Findings                        27296 non-null object
Therapies                       27296 non-null object
Immediate Complications         27296 non-null object
Post Procedure Complications    1 non-null object
Hospital                        27296 non-null object
Priority                        27296 non-null object
dtypes: datetime64[ns](2), int64(1), object(12)
memory usage: 3.1+ MB


In [14]:
df = df.drop_duplicates() # I know there are some duplicates in here so let's drop them, as you would in excel if you weren't sure

In [15]:
df.shape # This now shows us that the dataframe is 23837 rows by 15 columns

(23837, 15)

In [16]:
list_labels = ['Hospital Number', 'NHS Number', 'Patient Name', 'Date of Birth', 'Procedure', 'Endoscopist', 'Procedure Date', 'Age', 
       'Indications', 'Findings', 'Therapies', 'Immediate Complications', 'Post Procedure Complications' , 'Hospital', 'Priority'] 

In [17]:
df.columns = list_labels # This is how to assign the 'keys' to the columns - Note you don't have to do this. I'm just showing you how to do it in case the labels are wrong

# Extracting key data and anonymising records using Regex

What is a Regex I hear you say... Regex's are your best friend - say I wanted to extract all NHS numbers matching a pattern:

In [18]:
import re

In [19]:
NHS = re.compile('\d{3}-\d{3}-\d{4}')

I can use this to 'test' whether a number is an NHS number like so:

In [20]:
NHS_result = NHS.match('365-062-6578')
print(bool(NHS_result))

True


The same can easily be done in this dataset to work out in this case who are your Mr's and by default who are your Mrs/Miss/Ms

In [21]:
sex_pattern = '^MR\s'

In [22]:
matches = re.findall(sex_pattern, 'MR ')
print(matches)

['MR ']


In [23]:
df['Male'] = df['Patient Name'].str.contains(sex_pattern) # This stores the anonymised sex information in another column

In [24]:
df.Male.sum()

11726

Now we need to drop all the non-anonymised columns. The key principle is reduction of risk of re-indification. If you make it impossible without a key (the original dataset) to re-indentify the patients then you have made the data safe. This should always be a key guiding principle for you

In [25]:
df['Patient'] = pd.Categorical(df['Patient Name']) # this changes the datatype to categorical data-type

In [26]:
df['Patient Code'] = df['Patient'].cat.codes # this creates an anonymised column

In [27]:
df['NHS'] = pd.Categorical(df['NHS Number']) # this will store the values as categorical data-types

In [28]:
df['NHS code'] = df['NHS'].cat.codes # you can use either [] or .notation

In [29]:
df.info() # ahh its grown to 20 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23837 entries, 0 to 27295
Data columns (total 20 columns):
Hospital Number                 23837 non-null object
NHS Number                      23755 non-null object
Patient Name                    23837 non-null object
Date of Birth                   23837 non-null datetime64[ns]
Procedure                       23837 non-null object
Endoscopist                     23837 non-null object
Procedure Date                  23837 non-null datetime64[ns]
Age                             23837 non-null int64
Indications                     23837 non-null object
Findings                        23837 non-null object
Therapies                       23837 non-null object
Immediate Complications         23837 non-null object
Post Procedure Complications    1 non-null object
Hospital                        23837 non-null object
Priority                        23837 non-null object
Male                            23837 non-null bool
Patient           

In [30]:
df['Patient'].nunique() # counts the unique values in the column

18489

In [31]:
df['Patient Code'].nunique() # See the original non-anonymised string values have been converted to unique ints

18489

Now we have created entirely anonymised columns we can drop the patient sensitive data - note: This is not a good idea if you want to perform a merge operation requiring the keys, but we are not going to be doing this today and we want to handle anonymised data only for the purposes of this.

In [32]:
df.drop('Date of Birth', axis=1, inplace=True) # Bye bye hospital numbers

In [33]:
df.drop('Hospital Number', axis=1, inplace=True) # Bye bye hospital numbers

In [34]:
df.drop('Endoscopist', axis=1, inplace=True) # Bye bye endoscopists

In [35]:
df.drop('NHS Number', axis=1, inplace=True) # Bye bye NHS Numbers

In [36]:
df.drop('NHS', axis=1, inplace=True) # Bye bye NHS Numbers

In [37]:
df.drop('Hospital', axis=1, inplace=True) # We don't need this column either. If you don't need it drop it

In [38]:
df.drop('Patient', axis=1, inplace=True) # Finally we need to remove the patient names

In [39]:
df.drop('Patient Name', axis=1, inplace=True) # Finally we need to remove the patient names

In [40]:
df.head() # Now we can safely display the top 5 values

,Procedure,Procedure Date,Age,Indications,Findings,Therapies,Immediate Complications,Post Procedure Complications,Priority,Male,Patient Code,NHS code
0,Colonoscopy,2016-04-12 09:17:00,58,Abdominal mass,(1) Sigmoid colon: Polyp - flat (max_size 5 m...,None recorded,None,NaN,Routine,False,14010,13458
1,Colonoscopy,2011-01-10 14:41:00,85,Abdominal mass,(1) Sigmoid colon: Stricture - probably malign...,Sigmoid colon: Cold biopsy,None,NaN,Routine,True,4867,2690
2,Colonoscopy,2012-11-16 10:33:00,50,Abdominal mass,All normal or not visualised,None recorded,None,NaN,Routine,False,11873,14501
3,Colonoscopy,2015-06-20 10:50:00,46,Abdominal mass,All normal or not visualised,"Rectum: Cold biopsy (distance 10cm, specimen r...",None,NaN,Routine,False,10575,6084
4,Colonoscopy,2014-11-27 16:00:00,50,Abdominal mass,All normal or not visualised,None recorded,None,NaN,Routine,False,14890,13561


In [41]:
df.info() # there is still one pesky, almost useless column in there now

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23837 entries, 0 to 27295
Data columns (total 12 columns):
Procedure                       23837 non-null object
Procedure Date                  23837 non-null datetime64[ns]
Age                             23837 non-null int64
Indications                     23837 non-null object
Findings                        23837 non-null object
Therapies                       23837 non-null object
Immediate Complications         23837 non-null object
Post Procedure Complications    1 non-null object
Priority                        23837 non-null object
Male                            23837 non-null bool
Patient Code                    23837 non-null int16
NHS code                        23837 non-null int16
dtypes: bool(1), datetime64[ns](1), int16(2), int64(1), object(7)
memory usage: 1.9+ MB


As you can see we have now got a 12 column dataframe, but Post-Procedure Complications is almost useless so lets drop it

In [42]:
df.drop('Post Procedure Complications', axis=1, inplace=True) # Finally we need to remove the patient names

In [43]:
df.info() # We now have a beautifully complete pandas.core.frame.dataframe!! with 11 columns and 23837 rows. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23837 entries, 0 to 27295
Data columns (total 11 columns):
Procedure                  23837 non-null object
Procedure Date             23837 non-null datetime64[ns]
Age                        23837 non-null int64
Indications                23837 non-null object
Findings                   23837 non-null object
Therapies                  23837 non-null object
Immediate Complications    23837 non-null object
Priority                   23837 non-null object
Male                       23837 non-null bool
Patient Code               23837 non-null int16
NHS code                   23837 non-null int16
dtypes: bool(1), datetime64[ns](1), int16(2), int64(1), object(6)
memory usage: 1.8+ MB


Note that it contains different datatypes - objects, datetimes, integers, no floats at the moment

# This will drastically reduce the risk of re-indentification in your dataset

If you want to take this to the next level go to: https//www.clinicaldevelopers.com now!